In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1 import ImageGrid, AxesGrid, make_axes_locatable, SubplotDivider

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../src'))

from figure_presets import *
from plotting_functions import *

from adaptive_response.adaptive_threshold import AdaptiveThresholdNumeric
from utils.numba.tools import random_seed

In [ ]:
Nr = 32
Nl = 256
alpha = 1.4
s = 0.1 * Nl
width = 1

In [ ]:
parameters = {'c_distribution': 'log-normal'}

model = AdaptiveThresholdNumeric(Nl, Nr, parameters=parameters)
model.threshold_factor = alpha
model.choose_commonness('const', mean_mixture_size=s)
model.c_means = 1
model.c_vars = 1


init_state = model.parameters['initialize_state']
init_state['c_mean'] = 'exact'
init_state['c_var'] = 'exact'
init_state['correlations'] = 'exact'

In [ ]:
random_seed(4)

model.choose_sensitivity_matrix('log-normal', mean_sensitivity=1, width=width)


ymax = 32

with figure_file(
            'excitation_pattern.pdf',
            fig_width_pt=200., crop_pdf=False, legend_frame=False,
            transparent=True, post_process=False, num_ticks=(6, 6)
        ) as fig:
    
    en_plot = next(model._sample_excitations(1))

    en = en_plot.copy()

    xs = np.arange(len(en)) + 1
    bars = plt.bar(xs - 0.5, en, width=1,
                   color=COLOR_BLUE, edgecolor='none', lw=0)

    plt.axhline(alpha * en.mean(), color=COLOR_RED)
    #plt.axhline(en.mean(), color='k')

    for i in np.flatnonzero(en > alpha * en.mean()):
        bars[i].set_color(COLOR_ORANGE)

    plt.xlim(0.5, len(en) + 0.5)
    plt.ylim(0, ymax)
    #plt.yticks(np.arange(0, ymax, 20))

    plt.ylabel('Excitation $e_n$')
    plt.xlabel('Channel $n$');

    ax_right = plt.gca().twinx()
    ax_right.set_ylim(0, ymax/en.mean())
    ax_right.set_yticks([0,1,2])
    #ax_right.set_ylabel('Norm. Exc. $\hat e_n$')
    
print('Finished')